# Árvore de Decisão

Ao se utilizar sklearn para implementar uma árvore de decisão, apenas chamamos os métodos, mas não implementamos o algorítmo do zero.

Tentarei construir esse jupyter notebook focando em discutir a matemática e as estatísticas fundamentais por trás de um modelo de algoritmo de Árvore de Decisão.O objetivo é entender, em um nível mais baixo, como a Árvore de Decisão funciona em segundo plano.

A Árvore de Decisão funciona no princípio das condições. O algoritmo verifica as condições, em um nó, e divide os dados, conforme o resultado, da instrução condicional. O algoritmo de árvore de decisão pertence à família de algoritmos de aprendizado de máquina supervisionado. Ele pode ser usado para construir classificações, bem como modelos de regressão.

### Termos essenciais

**Entropia** - A entropia de um conjunto de dados, é uma medida da impureza do conjunto de dados. A entropia também pode ser considerada, como uma medida de incerteza. Devemos tentar minimizar, a entropia. O objetivo dos modelos de aprendizado de  máquina é reduzir a incerteza ou a entropia, tanto quanto possível.

**Ganho de informação** - o ganho de informação, é uma medida de quanta informação, uma característica nos dá sobre as aulas. O algoritmo de Árvores de Decisão sempre tentará maximizar o ganho de informação. Recurso, que particiona perfeitamente os dados, deve fornecer o máximo de informações. Um recurso, com o maior ganho de informação, será usado para dividir primeiro.

## Implementação

In [1]:
#Importação das bibliotecas
import pandas as pd
import numpy as np

In [2]:
#Utilizando pandas para carregar um conjunto de dados em um dataframe
df = pd.read_csv('data-dt.csv')
df.head()

,Glucose,BloodPressure,Diabetes
0,86,104,1
1,78,111,0
2,79,114,0
3,77,105,0
4,90,100,1


In [3]:
#Definição da função de cálculo de entropia
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

**Cálculo de ganho de informação**: esta função está usando três parâmetros, dataset, feature e label. Primeiro calculamos a entropia do conjunto de dados. Então, calculamos a entropia do recurso ponderado. Observe que, estamos chamando, a função *calculate_entropy*, de dentro desta função. No final desta função, somos capazes de calcular *feature_information_gain* (característica do ganho de informação), que é igual a *dataset_entropy* (entropia do dataset) menos *weighted_feature_entropy* (entropia do recurso ponderado).

In [4]:
#Definição da função de cálculo do ganho da informação
def calculate_information_gain(dataset, feature, label):
    #Calcula a entropia do dataset
    dataset_entropy = calculate_entropy(dataset[label])
    values, feat_counts = np.unique(dataset[feature], return_counts = True)
    
    #Cálculo da entropia do recurso ponderado                               #Chamada da função calculate_entropy
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                               ==values[i]).dropna()[label]) for i in range(len(values))])
    feature_info_gain = dataset_entropy - weighted_feature_entropy
    return feature_info_gain

**Função de criação de árvore de decisão**: Uma árvore de decisão, recursivamente, divide o conjunto de treinamento em subconjuntos cada vez menores. As árvores de decisão são treinadas, passando os dados de um nó raiz para as folhas. Os dados são divididos repetidamente, de acordo com as variáveis de recurso, para que os nós filhos sejam mais puros. Isso ajuda a reduzir a entropia. Um nó puro significa que todas as amostras nesse nó têm o mesmo rótulo.

In [5]:
#Definição da função de criação da árvore de decisão
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label], return_counts=True)
    unique_data = np.unique(dataset[label])
    
    if len(unique_data) <= 1:
        return unique_data[0]
    
    elif len(dataset) == 0 :
        return unique_data[np.argmax(datum[1])]
    
    elif len(features) == 0:
        return parent
    
    else:
        parent = unique_data[np.argmax(datum[1])]
        
        #Chama a função calculate_information_gain
        item_values = [calculate_information_gain(dataset, feature, label) for feature in features]
        
        optimum_feature_index = np.argmax(item_values)
        optimum_feature = features[optimum_feature_index]
        decision_tree = {optimum_feature:{}}
        features = [i for i in features if i != optimum_feature]
        
        for value in np.unique(dataset[optimum_feature]):
            min_data = dataset.where(dataset[optimum_feature] == value).dropna()
            
            #Chamada recursiva para a função create_decision_tree
            min_tree = create_decision_tree(min_data, df, features, label, parent)
            
            decision_tree[optimum_feature][value] = min_tree
            
        return(decision_tree)

In [6]:
#Definição da função de predição do tipo de diabetes
def predict_diabetes(test_data, decision_tree):
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        decision_tree = decision_tree[nodes][value]
        
        prediction = 0
        if type(decision_tree) is dict:
            #Chamada recursiva para a função predict_diabetes
            prediction = predict_diabetes(test_data, decision_tree)
        else:
            prediction = decision_tree
            break;
            
    return prediction

In [9]:
# Define os recursos e o rótulo
features = df.columns[:-1]
label = 'Diabetes'
parent=None
features

Index(['Glucose', 'BloodPressure'], dtype='object')

In [10]:
#Treina o modelo da árvore de decisão
decision_tree = create_decision_tree(df, df, features, label, parent)

In [11]:
#Efetua a predição utilizando o modelo treinado
sample_data = {'Glucose':86,'BloodPressure':104}
test_data = pd.Series(sample_data)
prediction = predict_diabetes(test_data,decision_tree)
prediction

1.0